# A3
* GCR = 0.4
* Albedo = 0.2
* Hub Height = 3.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* A GCR of 0.4 for the Trina Vertex module (length = 2.384m) corresponds to a rowto-row spacing or pitch of 5.96m
* 1-up Portrait: 5 rows, each with 25 modules
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/A3'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/A3


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+237.g204310a.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'A3'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1 # m
module_y = 2 # m. slope we will measure
sensorsy=2
torquetube = 0.15

# SceneDict Parameters
pitch = 5.96 # m
albedo = 0.2
hub_height = 3.5 # m  
nMods = 25 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/A3
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [6]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y)


Module Name: PVmod
Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [7]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [8]:
file = 'Phase2_meteo_hourly_psm3format.csv'
directory = os.path.expanduser('~/Documents/IEA-Bifacial-Tracking-Modeling')
weatherfile = os.path.join(directory, file)

metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-01-01_1000', endtime='2022-01-01_1200')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [9]:
trackerdict = demo.set1axis(**trackerParams)

In [10]:
if cumulativesky:
    demo.genCumSky1axis()
else:
    demo.gendaylit1axis()

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/


In [11]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/

Making 2 octfiles in root directory.
Created 1axis_2022-01-01_1030.oct
Created 1axis_2022-01-01_1130.oct


In [12]:
trakerdict = demo.analysis1axis(sensorsy=2)

Linescan in process: 1axis_2022-01-01_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-01-01_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-01-01_1030_Row3_Module13.csv
Index: 2022-01-01_1030. Wm2Front: 101.58905. Wm2Back: 18.268410000000003
Linescan in process: 1axis_2022-01-01_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-01-01_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-01-01_1130_Row3_Module13.csv
Index: 2022-01-01_1130. Wm2Front: 262.3367. Wm2Back: 36.81952


In [13]:
trakerdict

{'2022-01-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 29.16,
  'theta': -29.16,
  'dni': 5,
  'ghi': 112,
  'dhi': 108,
  'temp_air': 3.9,
  'wind_speed': 4.7,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_1030.rad',
  'radfile': 'objects/1axis2022-01-01_1030__C_3.01275_rtr_5.96000_tilt_29.16000_25modsx5rows_origin0,0.rad',
  'scene': {'module': {'x': 1, 'y': 2, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.01, 'sceney': 2.0, 'scenez': 0.1, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmod 1 2 0.02 | xform -t -0.5 -1.0 0 -a 1 -t 0 2.0 0', 'modulefile': 'objects/PVmod.rad', 'glass': False, 'offsetfromaxis': 0, 'xgap': 0.01, 'ygap': 0.0, 'zgap': 0.1}, 'modulefile': 'objects/PVmod.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.33557046979865773, 'text': '!xform -rx 29.16 -t 0 0 3.5 -a 25 -t 1.01 0 0 -a 5 -t 0 5.96 0 -i 1 -t -12.120000000000001 -11.92 0 -rz 90.0 -t 0 0 0 objects/PVmod.rad', 'radfiles': 'objects/1axis2022-01-01_1030__C_3.01275_rtr_5.96000_tilt_29.16000_25mods

In [14]:
demo

{'metdata': {'latitude': 35.05, 'longitude': -106.54, 'elevation': 1600.0, 'timezone': -7.0, 'city': 'Albuquerque', 'datetime': [Timestamp('2022-01-01 10:30:00-0700', tz='Etc/GMT+7'), Timestamp('2022-01-01 11:30:00-0700', tz='Etc/GMT+7')], 'ghi': array([112, 265]), 'dhi': array([108, 232]), 'dni': array([ 5, 60]), 'albedo': None, 'dewpoint': None, 'pressure': None, 'temp_air': array([3.9, 3.3]), 'wind_speed': array([4.7, 5.5]), 'meastracker_angle': None, 'solpos':                            apparent_zenith     zenith  apparent_elevation  \
corrected_timestamp                                                         
2022-01-01 11:00:00-07:00        60.301624  60.325832           29.698376   
2022-01-01 12:00:00-07:00        58.030863  58.053009           31.969137   

                           elevation     azimuth  equation_of_time  
corrected_timestamp                                                 
2022-01-01 11:00:00-07:00  29.674168  161.465455         -3.652581  
2022-01-01 12:0

In [15]:
demo.calculateResults()

Bifaciality factor of module stored is  1
No CECModule data passed; using default for Prism Solar BHC72-400


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

{'2022-01-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 29.16,
  'theta': -29.16,
  'dni': 5,
  'ghi': 112,
  'dhi': 108,
  'temp_air': 3.9,
  'wind_speed': 4.7,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_1030.rad',
  'radfile': 'objects/1axis2022-01-01_1030__C_3.01275_rtr_5.96000_tilt_29.16000_25modsx5rows_origin0,0.rad',
  'scene': {'module': {'x': 1, 'y': 2, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.01, 'sceney': 2.0, 'scenez': 0.1, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmod 1 2 0.02 | xform -t -0.5 -1.0 0 -a 1 -t 0 2.0 0', 'modulefile': 'objects/PVmod.rad', 'glass': False, 'offsetfromaxis': 0, 'xgap': 0.01, 'ygap': 0.0, 'zgap': 0.1}, 'modulefile': 'objects/PVmod.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.33557046979865773, 'text': '!xform -rx 29.16 -t 0 0 3.5 -a 25 -t 1.01 0 0 -a 5 -t 0 5.96 0 -i 1 -t -12.120000000000001 -11.92 0 -rz 90.0 -t 0 0 0 objects/PVmod.rad', 'radfiles': 'objects/1axis2022-01-01_1030__C_3.01275_rtr_5.96000_tilt_29.16000_25mods

In [16]:
demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

,timestamp,module,row,POA_eff,Grear_mean,Gfront_mean,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI
0,2022-01-01_1030,13,3,119.85746,18.26841,101.58905,49.833083,42.074555,17.982657,18.439953,0.000044,49.833061,4.7,5,108,112
1,2022-01-01_1130,13,3,299.15622,36.81952,262.33670,126.029194,110.525713,14.035215,14.027036,0.000002,126.029191,5.5,60,232,265
